<a href="https://colab.research.google.com/github/cdiegor/Simulacao/blob/main/Geracao_Variaveis_Aleatorias_Discretas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Geração de Variáveis Aleatórias (Distribuições Discretas) —



### Motores Categóricos & Validação


## ⚙️ Ambiente e Utilitários

In [ ]:
# =========================
# Utilitários: RNG, testes, bench
# =========================
import numpy as np
import time
from math import sqrt

def make_rng(seed=None):
    """Cria um Generator (PCG64) reprodutível."""
    return np.random.Generator(np.random.PCG64(seed))

def chisq_discrete(samples, support, pmf):
    """Retorna (estatística, gl, p) do teste qui-quadrado para suporte discreto.
    'support': array 1D com valores possíveis; 'pmf': probabilidades que somem 1.
    Observação: para células com esperado < 5, o p-valor pode ficar inválido; rebinning recomendado.
    """
    samples = np.asarray(samples)
    support = np.asarray(support)
    pmf = np.asarray(pmf, dtype=float)
    n = samples.size
    # contagens observadas
    key_to_idx = {v: i for i, v in enumerate(support)}
    counts = np.zeros_like(pmf, dtype=int)
    for x in samples:
        i = key_to_idx.get(x)
        if i is not None:
            counts[i] += 1
    expected = n * pmf
    obs, expd = counts.astype(float), expected.astype(float)
    # checagem simples de células raras
    if np.any(expd < 5):
        stat = np.sum((obs - expd)**2 / np.maximum(expd, 1e-12))
        df = max(len(pmf) - 1, 1)
        p = np.nan  # rebin necessário para p adequado
        return stat, df, p
    stat = np.sum((obs - expd)**2 / expd)
    df = len(pmf) - 1
    try:
        from scipy.stats import chi2
        p = chi2.sf(stat, df)
    except Exception:
        # aproximação normal para qui-quadrado
        z = (stat - df) / sqrt(2 * df)
        from math import erf
        p = 0.5 * (1 - erf(z / sqrt(2)))
    return stat, df, p

def check_moments(samples, mean_true, var_true):
    """Compara média/variância empíricas com valores teóricos."""
    x = np.asarray(samples)
    mean_emp = x.mean()
    var_emp = x.var(ddof=1)
    return mean_emp, var_emp, mean_emp - mean_true, var_emp - var_true

def time_draw(fn, *args, repeat=3, **kwargs):
    """Cronometra a função geradora, retornando (resultado, melhor_tempo_s)."""
    best = float('inf')
    out = None
    for _ in range(repeat):
        t0 = time.perf_counter()
        out = fn(*args, **kwargs)
        dt = time.perf_counter() - t0
        best = min(best, dt)
    return out, best

## 🧮 Categórica — Inversa & Alias

In [ ]:
# =========================
# Categórica: CDF inversa (busca binária) e Alias (Walker–Vose)
# =========================
def categorical_inverse(p, size, rng=None):
    """Amostra 'size' valores de uma distribuição categórica com probs p (CDF inversa).
    Retorna índices inteiros em [0, K-1].
    """
    rng = make_rng() if rng is None else rng
    p = np.asarray(p, dtype=float)
    p = p / p.sum()
    cdf = np.cumsum(p)
    u = rng.random(size)
    return np.searchsorted(cdf, u, side='left')

class AliasTable:
    """Tabela de alias de Walker–Vose para amostragem O(1) após preparo O(K)."""
    def __init__(self, p):
        p = np.asarray(p, dtype=float)
        if p.sum() <= 0:
            raise ValueError("As probabilidades devem somar > 0")
        p = p / p.sum()
        K = len(p)
        self.prob = np.zeros(K)
        self.alias = np.zeros(K, dtype=int)
        q = p * K
        small = [i for i, x in enumerate(q) if x < 1.0]
        large = [i for i, x in enumerate(q) if x >= 1.0]
        while small and large:
            i = small.pop()
            j = large.pop()
            self.prob[i] = q[i]
            self.alias[i] = j
            q[j] = q[j] - (1.0 - q[i])
            if q[j] < 1.0:
                small.append(j)
            else:
                large.append(j)
        for j in large + small:
            self.prob[j] = 1.0
            self.alias[j] = j

    def sample(self, size, rng=None):
        rng = make_rng() if rng is None else rng
        K = len(self.prob)
        i = (rng.random(size) * K).astype(int)
        u = rng.random(size)
        take = u < self.prob[i]
        out = np.where(take, i, self.alias[i])
        return out

## 🎯 Geométrica & NegBin

In [ ]:
# =========================
# Geométrica (duas parametrizações) e NegBin (mistura Poisson–Gama)
# =========================
def geometric_trials(p, size, rng=None):
    """Geom(p) em {1,2,...}: número de tentativas até o 1º sucesso."""
    rng = make_rng() if rng is None else rng
    u = rng.random(size)
    return 1 + np.floor(np.log1p(-u) / np.log1p(-p)).astype(int)

def geometric_failures(p, size, rng=None):
    """Geom(p) em {0,1,...}: número de falhas antes do 1º sucesso."""
    rng = make_rng() if rng is None else rng
    u = rng.random(size)
    return np.floor(np.log1p(-u) / np.log1p(-p)).astype(int)

def negbin(r, p, size, rng=None):
    """NegBin(r,p) via mistura Poisson–Gama (r > 0 não precisa ser inteiro)."""
    rng = make_rng() if rng is None else rng
    lam = rng.gamma(shape=r, scale=(1-p)/p, size=size)
    return rng.poisson(lam)

## 🔢 Poisson

In [ ]:
# =========================
# Poisson: Knuth (λ pequeno) + wrapper
# =========================
def poisson_knuth(lam, size, rng=None):
    """Método do produto de Knuth; eficiente para λ <= ~30."""
    rng = make_rng() if rng is None else rng
    lam = float(lam)
    L = np.exp(-lam)
    out = np.empty(size, dtype=int)
    for i in range(size):
        k = 0
        t = 1.0
        while t > L:
            k += 1
            t *= rng.random()
        out[i] = k - 1
    return out

def poisson_any(lam, size, rng=None):
    """Wrapper: usa Knuth para λ<=30 e NumPy para λ maior (didático)."""
    rng = make_rng() if rng is None else rng
    lam = float(lam)
    if lam <= 30:
        return poisson_knuth(lam, size, rng)
    return rng.poisson(lam, size)

## 🧰 Binomial

In [ ]:
# =========================
# Binomial: inversão via recorrência + wrappers
# =========================
from math import sqrt

def binomial_inv(n, p, size, rng=None):
    """Inversão por recorrência estável (laboratório; n moderado).
    Custo ~ O(n) por amostra quando p ~ 0.5 (atravessar a CDF).
    """
    rng = make_rng() if rng is None else rng
    n = int(n); p = float(p)
    q = 1.0 - p
    out = np.empty(size, dtype=int)
    for i in range(size):
        u = rng.random()
        k = 0
        w = q**n  # P(X=0)
        c = w
        while u > c and k < n:
            k += 1
            w *= (n - (k - 1)) / k * (p / q)
            c += w
        out[i] = k
    return out

def binomial_normal_approx(n, p, size, rng=None):
    rng = make_rng() if rng is None else rng
    mu = n * p
    sigma = sqrt(n * p * (1 - p))
    x = rng.normal(mu, sigma, size)  # com correção de continuidade ao arredondar
    return np.clip(np.round(x), 0, n).astype(int)

def binomial_poisson_approx(n, p, size, rng=None):
    rng = make_rng() if rng is None else rng
    lam = n * p
    return rng.poisson(lam, size)

def binomial_any(n, p, size, rng=None):
    """Wrapper didático de regime: usa inversão em n moderado e NumPy no geral."""
    n = int(n); p = float(p); rng = make_rng() if rng is None else rng
    if n <= 20000 and min(n*p, n*(1-p)) <= 1500:
        return binomial_inv(n, p, size, rng)
    return rng.binomial(n, p, size)

## 🎲 Hipergeométrica & Uniforme Discreta

In [ ]:
# =========================
# Hipergeométrica (inversão por razão) e Uniforme Discreta (sem viés)
# =========================
def hypergeom_inv(N, K, n, size, rng=None):
    """Inversão via recorrência de razões para HGe(N,K,n).
    Retorna #sucessos em n retiradas sem reposição (uso didático; tamanhos moderados).
    """
    rng = make_rng() if rng is None else rng
    N = int(N); K = int(K); n = int(n)
    kmin = max(0, n - (N - K))
    kmax = min(n, K)
    out = np.empty(size, dtype=int)
    for i in range(size):
        u = rng.random()
        k = kmin
        w = 1.0  # peso relativo inicial
        c = w
        while u > c and k < kmax:
            r = ((K - k) * (n - k)) / ((k + 1) * (N - K - n + k + 1))
            w *= r
            c += w
            k += 1
        out[i] = k
    return out

def discrete_uniform(a, b, size, rng=None):
    """Uniforme discreta não-viesada em {a,...,b} (método do módulo com rejeição)."""
    rng = make_rng() if rng is None else rng
    a = int(a); b = int(b)
    R = b - a + 1
    out = np.empty(size, dtype=int)
    for i in range(size):
        while True:
            z = rng.integers(0, 1 << 64, dtype=np.uint64)
            t = ((1 << 64) // R) * R
            if z < t:
                out[i] = a + int(z % R)
                break
    return out

## 📊 Dicas de Gráficos (matplotlib)
- Use **apenas `matplotlib`**, um gráfico por figura;
- Não fixe cores manualmente (padrão é suficiente);
- Exemplos abaixo geram histogramas e comparações empírico vs teórico.

In [ ]:
import matplotlib.pyplot as plt

def plot_categorical_empirical_vs_theoretical(samples, p):
    p = np.asarray(p, dtype=float); p = p / p.sum()
    K = len(p)
    counts = np.bincount(samples, minlength=K)
    emp = counts / counts.sum()
    idx = np.arange(K)
    plt.figure()
    plt.bar(idx - 0.2, emp, width=0.4, label="Empírico")
    plt.bar(idx + 0.2, p, width=0.4, label="Teórico")
    plt.xlabel("Categoria")
    plt.ylabel("Probabilidade")
    plt.title("Categórica: Empírico vs Teórico")
    plt.legend()
    plt.show()

def plot_histogram_discrete(samples, title="Histograma (discreto)"):
    vals, counts = np.unique(samples, return_counts=True)
    plt.figure()
    plt.bar(vals, counts / counts.sum())
    plt.xlabel("Valor")
    plt.ylabel("Frequência relativa")
    plt.title(title)
    plt.show()

## 🧪 Exemplos Rápidos (execute as células)

In [ ]:
# RNG base
rng = make_rng(2025)

# 1) Categórica: inversa vs alias
p = np.array([0.05, 0.10, 0.15, 0.20, 0.50])
alias = AliasTable(p)

x_inv, t_inv = time_draw(categorical_inverse, p, 200_000, rng=rng)
x_alias, t_alias = time_draw(alias.sample, 200_000, rng=rng)
print(f"Inversa: {t_inv:.4f}s | Alias: {t_alias:.4f}s")

# Validação: qui-quadrado e gráfico
stat, df, pval = chisq_discrete(x_alias, np.arange(len(p)), p)
print(f"Qui-quadrado (alias): stat={stat:.2f}, gl={df}, p={pval}")
plot_categorical_empirical_vs_theoretical(x_alias, p)

# 2) Poisson em vários λ
for lam in [1, 5, 20, 100]:
    x, t = time_draw(poisson_any, lam, 200_000, rng=rng)
    mean_emp, var_emp, dmu, dvar = check_moments(x, lam, lam)
    print(f"Pois({lam}): t={t:.4f}s | média={mean_emp:.3f} (Δ{dmu:.3f}) | var={var_emp:.3f} (Δ{dvar:.3f})")
    if lam <= 20:
        plot_histogram_discrete(x, title=f"Poisson({lam})")

# 3) Binomial (n moderado) e checagem de momentos
n, pbin = 2000, 0.3
x, t = time_draw(binomial_any, n, pbin, 200_000, rng=rng)
mu = n * pbin; var = n * pbin * (1 - pbin)
mean_emp, var_emp, dmu, dvar = check_moments(x, mu, var)
print(f"Bin({n},{pbin}): t={t:.4f}s | média={mean_emp:.2f} (Δ{dmu:.2f}) | var={var_emp:.2f} (Δ{dvar:.2f})")